### Imports

In [15]:
import gurobipy as gp
import pandas as pd
import numpy as np
import winsound
import os

# from IPython.testing.globalipapp import get_ipython
# from IPython.core.display import display

%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


### Functions

In [16]:
def vprint_factory(verbose: bool=False):
    if verbose:
        return print
    return lambda *x, **y: None

In [17]:
def extract(data, row, fact, maximize=True):
    text = str(data.iloc[row][fact]).strip("mg%?").replace(",", ".")
    if str(text) == "nan" or len(text) == 0:
        raise ValueError(f"Row {row}, Fact {fact}, Max {maximize} has no value")  # This may not be an error, but it may be a potential source
        return maximize * 1_000_000
    return float(text)

In [18]:
def classify_age(a: int, year=2020) -> str:
    a = round(a, 0)
    age_strings_2015 = ["1-3", "4-8", "9-13", "14-18", "19-30", "31-50", "51+"]
    age_strings_2020 = ["2-3", "4-8", "9-13", "14-18", "19-30", "31-50", "51+"]
    age_groups = [tuple([int(age) for age in i.split("-")]) if "-" in i else (int(i.strip("+")), 110) for i in age_strings_2015]
    for i, age in enumerate(age_groups):
        if age[0] <= a <= age[1]:
            if year == 2020:
                return age_strings_2020[i]
            else:
                return age_strings_2015[i]
    else:
        raise ValueError(f"{a} is not a valid age in the {year} guidelines.")

### File Input

In [42]:
def load_ref_files(nutrition_name="Nutritional Facts - Categorized", guidelines_name="Dietary Guidelines", nutrition_cols=[], year=2020):
    df = pd.read_excel(nutrition_name + ".xlsm")
    guide = pd.read_excel(guidelines_name + ".xlsx", header=1,
                          index_col=[1, 2], skiprows=[2],
                          sheet_name=f"Dietary Guidelines {year}").drop("Unnamed: 0", axis=1)
    
    # Cleaning out commas
    comma_problem_numeric_cols = ("Serving Size", "Saturated Fat", "Trans Fat")
    remove_commas = lambda s: str(s).replace(",", ".")
    df.loc[:, comma_problem_numeric_cols] = df.loc[:, comma_problem_numeric_cols].applymap(remove_commas)
    
    # Swapping Godfather's Pizza Drinks Calories and Protein values
    incorrect_sugars = df[(df["Restaurant"] == "Godfather's Pizza") & (df["Common Category"] == "Beverages")]
    basic_float_conversion = lambda x: float(x.strip("g"))
    incorrect_sugars.loc[:, "Sugars"] = ((incorrect_sugars["Dietary Fiber"].map(basic_float_conversion)
                                          + incorrect_sugars["Protein"].map(basic_float_conversion)) / 2).astype(str) + "g"
    incorrect_sugars.loc[:, ("Dietary Fiber", "Protein")] = "0g"
    df.loc[(df["Restaurant"] == "Godfather's Pizza") & (df["Common Category"] == "Beverages")] = incorrect_sugars
    
    # Stripping '=' from Jersey Mike's reported sodium values
    equals_removal = lambda x: str(x.split("=")[0])
    df.loc[df["Restaurant"] == "Jersey Mike's", "Sodium"] = df.loc[df["Restaurant"] == "Jersey Mike's", "Sodium"].map(equals_removal)
    
    # Detecting Uncategorized Items
    if np.nan in list(pd.unique(df["Common Category"])):
        raise ValueError("Some foods have no value for Common Category. Please run the Excel VBA script to assign.")
    
    # Removing Null Entries
    drop_index = pd.Series([False for _ in df.index])
    for col in nutrition_cols:
        drop_index = drop_index | (df[col] == "?")
    drop_index = drop_index[drop_index].index
    df.drop(drop_index, inplace=True)
    
    if "Unnamed: 24" in df.columns:
        df.drop("Unnamed: 24", axis=1, inplace=True)
        
    df = df.reindex()
    
    return df, guide

### Requirement Selection

In [43]:
def guide_lookup(gender: str, age: int, guide: pd.DataFrame, columns=[]):
    ff_nutrition_to_guidelines = {"Protein": "Protein (g)",
                                  "Vitamin A %": "Vitamin A (mcg RAEd)",
                                  "Sodium": "Sodium (mg)",
                                  "Total Carbohydrates": "Carbohydrate (g)",
                                  "Dietary Fiber": "Fiber (g)",
                                  "Calories": "Calorie Level Assessed", }
    guidelines_to_ff_nutrition = {value: key for key, value in ff_nutrition_to_guidelines.items()}
    guideline_kcals_to_ff_nutrition = {# "Total lipid (% kcal)": "Total Fat",
                                       "Added Sugars (% kcal)": "Sugars",
                                       "Saturated Fatty Acids (% kcal)": "Saturated Fat",
                                       "Calorie Level Assessed": "Calories From Fat"}
    genders = {"m": "Male", "f": "Female"}
    gender = genders[gender[0].lower()] # 'm' / 'M' / 'male' / 'Male' -> 'Male'
    # Selecting the appropriate Row
    filtered = guide.loc[gender, classify_age(age)]
    # Handling kcal measurements
    cal_level = filtered["Calorie Level Assessed"]
    nutrient_cals = {"Total Fat": 9, "Saturated Fat": 9, "Sugars": 4, "Calories From Fat": cal_level / 10}  # Cals from fat <= 10% cal_level
    for kcal_nutrient, out_name in guideline_kcals_to_ff_nutrition.items():
        new_entry = pd.Series(index=[out_name],
                              data=float(str(filtered[kcal_nutrient]).strip("<>").split("-")[-1]) / 100
                                         * cal_level / nutrient_cals[out_name])
        filtered = filtered.append(new_entry)
        filtered.drop(columns=kcal_nutrient, inplace=True)
    filtered.drop((col for col in filtered.index
                   if col not in guidelines_to_ff_nutrition.keys()
                   and col not in guideline_kcals_to_ff_nutrition.values()),
                  inplace=True)
    filtered.rename(guidelines_to_ff_nutrition, inplace=True) # I made this right at the end to potentially fix a problem, but I don't think it worked. Evaluate!
    return filtered[columns] if columns else filtered

### Model Building: Variables, Constraints, and Objective

In [55]:
def create_model(subset: pd.DataFrame, less_thans: list, equal_tos: list, guide: pd.DataFrame, costs: dict, objective: list,
                 filter_relaxations: set = {}, meals: int = 2, cat_limit: int = 0, min_cutoff: int = 1,
                 loose_equality: float = 0, var_type: str = "C", verbose: bool = False):
    """
    Generates the Gurobi model according to many available filters and options, discussed below
    
    :subset: menu items to consider, typically broken down by restaurant
    :less_thans: list of nutrients in guide where the sum of food nutritions must be less than the guideline (such as sodium) rather than more (protein)
    :guide: dietary recommendations taken from the government's '20-'25 Dietary Guidelines For Americans.
    :costs: dictionary of penalties associated with exceeding / falling short of the recommendations. Must include objective nutrients
    :objective: list of which nutrient(s) to rank by. Multiple items cause a multi-objective
    :filter_relaxations: recommendations in guide which may be relaxed. TODO: untested
    :meals: requires the solution to meet only (1 / meals) of each nutrient recommendation. Meals=1 & cat_limit=1 usually infeasible. Default: 2
    :cat_limit: requires no more than cat_limit of any one food type in the solution. Prevents 31 apple juice box solutions. Default: 0 (no limit)
    :min_cutoff: when non-zero, omits foods from the solution with fewer of the objective (calories, g sugar, mg sodium, etc.) than the cutoff * that nutrient's cost. Default: 1
    :loose_equality: float relaxing equalities [0-1] -> x +- loose_equality
    :var_type: decision variable type in B(inary), I(nteger), or C(ontinuous). Default: Continuous
    :verbose: provides detailed constraining / solving progress updates. Disable for more concise output. Default: False
    """
    vprint = vprint_factory(verbose)
    
    if type(objective) is not list:
        objective = [objective]
    if len(objective) > 1:
        for obj in objective:
            if obj not in costs.keys():
                raise ValueError(f"{obj} has no associated cost in costs: {costs}. This is required to form a multi-objective.")
        filter_relaxations.update(cost for cost in costs.keys() if cost in objective)  # Multiobjectives rely on relaxations and costs for their variables
    
    m = gp.Model()
    
    if (vtype := var_type[0].upper()) not in "BIC":
        raise ValueError(f"var_type must be one of B(inary), I(nteger), or C(ontinuous). '{vtype}' was passed")
    xis = [m.addVar(vtype=vtype) for _ in subset.index]  # Whether to include a food in the meal
    f_rel = {fact: m.addVar(name=f"{fact}_rel") if fact in filter_relaxations else 0 for fact in guide.index}  # excess variable
    
    for fact, req in guide.items():  # Nutrition Requirements
        if len(objective) == 1 and fact == objective[0]:  # Don't constrain the objective for single-objective models
            continue
        if (fact in less_thans) or (fact in equal_tos):
            scaled_req = req * (1 + (loose_equality if fact in equal_tos else 0))
            vprint(f"Constraining {fact}".ljust(35), f"<= {round(scaled_req, 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=True)
                             for r, x in enumerate(xis))) - f_rel[fact] <= scaled_req / meals)
        if (fact not in less_thans) or (fact in equal_tos):
            scaled_req = req * (1 - (loose_equality if fact in equal_tos else 0))
            vprint(f"Constraining {fact}".ljust(35), f">= {round(scaled_req, 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=False)
                             for r, x in enumerate(xis))) + f_rel[fact] >= scaled_req / meals)

    if min_cutoff:
        for obj in objective:
            for i, food in enumerate(subset.iterrows()):  # Excludes zero calorie (from fat) entries
                if extract(subset, i, obj, maximize=False) < min_cutoff * costs[obj]:  # If a fact is unknown, it's assumed to be -500,000
                    m.addConstr(xis[i] == 0)  # If a food has fewer than the min_cutoff of any objective, require 0 of it in the solution
                
    if cat_limit:
        vprint()
        for cat in pd.unique(subset["Common Category"]):
            vprint(f"Constraining only {cat_limit} or fewer {cat.strip('s')} items.")
            m.addConstr(sum((x for i, x in enumerate(xis) if subset["Common Category"][i] == cat)) <= cat_limit)
    
    m.setParam("OutputFlag", verbose)
    
    if len(objective) > 1:
        # The multiobjective value is the sum of the products of each's nutrients overage / shortage with its relative weight
        m.ModelSense = gp.GRB.MINIMIZE
        m.setObjective(sum(difference * costs[nutrient]
                           if type(difference) is gp.Var else 0
                           for nutrient, difference in f_rel.items() if nutrient in objective))
    else:
        # For less_than constraints, the nutrient ought to be minimized.
        # If data for an entry is unknown, it ought to be expensive for minimization
        # and negatively expensive for maximization as as to not be included in the solution
        m.ModelSense = gp.GRB.MINIMIZE if objective[0] in less_thans else gp.GRB.MAXIMIZE
        m.setObjective(sum((x * extract(subset, r, objective[0], maximize=-m.ModelSense) for r, x in enumerate(xis))) / meals)
    return m

In [56]:
def solution_data(model: gp.Model, model_args: dict, columns: list, multi_objective: bool) -> (pd.Series, pd.DataFrame):
    if model.status != 2:
        return None, None
    results = pd.Series(index=model_args["objective"], name=model_args["subset"].iloc[0]["Restaurant"], dtype="float64")
    
    if multi_objective:
        results.loc[model_args["objective"]] = [model.getVarByName(name + "_rel").x for name in model_args["costs"].keys()]
        results = results.append(pd.Series(model.ObjVal, index=["Overage Penalty"]))    
        choices = [var.x > 0 for var in model.getVars() if "_rel" not in var.varName]
        values = [var.x for var in model.getVars() if var.x > 0 and "_rel" not in var.varName]
    else:
        results.iloc[0] = model.ObjVal
        choices = [var.x > 0 for var in model.getVars()]
        values = [var.x for var in model.getVars() if var.x > 0]
    food_choice = model_args["subset"].loc[choices, ["Food"] + columns].reset_index(drop=True)
    food_choice.loc[:, "Amount"] = values
    return results, food_choice

In [57]:
def model_run(df: pd.DataFrame, objectives: list, multi_obj: bool, res: str="") -> pd.DataFrame:
    """
    Creates and runs models for each restaurant and objective, returning a pd.DataFrame of results and food selections
    :verbose:
    """
    vprint = vprint_factory(model_args["verbose"])
    model_args["verbose"] = (model_args["verbose"] - 1) == True  # Interesting verbosity logic. The True check is NOT unnecessary
    res_list = [res] if res else pd.unique(df["Restaurant"])
    master_results = pd.DataFrame(columns=objectives + (["Overage Penalty"] if multi_obj else []), index=res_list)
    master_foods = pd.DataFrame(columns=columns + ["Restaurant"] + (["Objective"] if not multi_obj else []), index=[])
    
    for res in res_list:
        vprint(res, end="\n\n")
        subset = df.loc[df["Restaurant"] == res].reset_index(drop=True)
        model_args["subset"] = subset

        if multi_obj:
            m = create_model(**model_args)
            m.optimize()
            results, foods = solution_data(m, model_args, columns, multi_obj)
        else:
            results = pd.DataFrame(columns=[], index=[res])
            foods = pd.DataFrame(columns=columns + ["Objective", "Restaurant"], index=[])
            for obj in objectives:
                vprint(f"Solving {res} with respect to {obj}")
                model_args["objective"] = [obj]
                m = create_model(**model_args)
                m.optimize()
                new_results, new_foods = solution_data(m, model_args, columns, multi_obj)
                if new_results is None:
                    vprint(f"{res}-{obj} is infeasible.")
                    new_results = pd.Series(index=[obj], name=res, dtype="float64")
                    results = pd.concat([results, pd.DataFrame(new_results).T], axis=1)
                    continue
                results = pd.concat([results, pd.DataFrame(new_results).T], axis=1)
                new_foods.insert(0, "Objective", obj)
                foods = pd.concat([foods, new_foods], axis=0).reset_index(drop=True)
        if foods is not None:
            foods.loc[:, "Restaurant"] = res
            master_results.loc[res] = results if multi_obj else results.iloc[0]
            master_foods = master_foods.append(foods, ignore_index = True)
        else:
            master_results.loc[res] = "-"

    food_col_order = ['Restaurant', 'Amount', 'Food', 'Sodium', 'Sugars', 'Calories From Fat', 'Calories',
                      'Protein', 'Total Carbohydrates', 'Dietary Fiber', 'Saturated Fat']
    if not multi_obj:
        food_col_order.insert(1, "Objective")
    master_foods = master_foods.reindex(columns = food_col_order)

    if multi_obj:
        master_results = master_results.reindex(columns=master_results.columns[[3, 0, 1, 2]])
        master_results = master_results.replace("-", np.nan).sort_values("Overage Penalty")
    master_results.replace(np.nan, "-", inplace=True)

    return master_results, master_foods

### Data Loading

In [58]:
columns = ['Sodium', 'Sugars', 'Calories From Fat', 'Calories', 'Protein',
           'Total Carbohydrates', 'Dietary Fiber', 'Saturated Fat']
df, guide = load_ref_files(nutrition_cols=columns)

C:\Users\zacha\Anaconda3\envs\NutritionResearch\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\zacha\Anaconda3\envs\NutritionResearch\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\zacha\Anaconda3\envs\NutritionResearch\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


### Run Config

In [63]:
age, gender = 24, "Male"
objectives = ["Sugars", "Sodium", "Calories From Fat"]  # List of objective(s). Must be list-like
multi_obj = False  # If True, thae model objective is the sum of objective overages. Otherwise, the model is solved for each nutrient separately
verbose = 1    # Whether to provide run / solve updates (level of verbosity = [0-2])
res = ""           # Leave blank to run all
out_name = "LooseEqualityCatRestriction3" # Leave blank to skip saving

filtered_guide = guide_lookup(gender, age, guide, columns)

less_thans = ["Sodium", "Sugars", "Saturated Fat", "Calories From Fat"]  # These correspond to columns from Fast Food Nutrition dataset
equal_tos = ["Calories"]
costs = {"Sugars": 1, "Sodium": 50, "Calories From Fat": 4}   # Note units: Sugars (g) vs Sodium (mg)
model_args = {"less_thans": less_thans, "equal_tos": equal_tos, "guide": filtered_guide,
              "filter_relaxations": set(), "objective": objectives, "costs": costs,
              "meals": 1, "cat_limit": 3, "min_cutoff": False, "loose_equality": 0,
              "var_type": "Continuous", "verbose": verbose, }

results, foods = model_run(df, objectives, multi_obj, res)
display(results)
display(foods)

if out_name:
    outname = "./ModelOutput/" + out_name + ("-Multi" if multi_obj else "") + (f"-{res}" if res else "") + ".xlsx"
    with pd.ExcelWriter(outname) as writer:
        results.to_excel(writer, sheet_name = "Rankings")
        foods.to_excel(writer, sheet_name = "Foods", index=False)
%notify -m "Run Completed"

Arby's

Solving Arby's with respect to Sugars
Arby's-Sugars is infeasible.
Solving Arby's with respect to Sodium
Arby's-Sodium is infeasible.
Solving Arby's with respect to Calories From Fat
Arby's-Calories From Fat is infeasible.
Baskin-Robbins

Solving Baskin-Robbins with respect to Sugars
Baskin-Robbins-Sugars is infeasible.
Solving Baskin-Robbins with respect to Sodium
Baskin-Robbins-Sodium is infeasible.
Solving Baskin-Robbins with respect to Calories From Fat
Baskin-Robbins-Calories From Fat is infeasible.
Blimpie

Solving Blimpie with respect to Sugars
Blimpie-Sugars is infeasible.
Solving Blimpie with respect to Sodium
Solving Blimpie with respect to Calories From Fat
Bojangles

Solving Bojangles with respect to Sugars
Bojangles-Sugars is infeasible.
Solving Bojangles with respect to Sodium
Bojangles-Sodium is infeasible.
Solving Bojangles with respect to Calories From Fat
Boston Market

Solving Boston Market with respect to Sugars
Boston Market-Sugars is infeasible.
Solving Bo

Shake Shack-Calories From Fat is infeasible.
Smashburger

Solving Smashburger with respect to Sugars
Smashburger-Sugars is infeasible.
Solving Smashburger with respect to Sodium
Smashburger-Sodium is infeasible.
Solving Smashburger with respect to Calories From Fat
Smashburger-Calories From Fat is infeasible.
Sonic

Solving Sonic with respect to Sugars
Solving Sonic with respect to Sodium
Sonic-Sodium is infeasible.
Solving Sonic with respect to Calories From Fat
Subway

Solving Subway with respect to Sugars
Solving Subway with respect to Sodium
Solving Subway with respect to Calories From Fat
Taco Bell

Solving Taco Bell with respect to Sugars
Taco Bell-Sugars is infeasible.
Solving Taco Bell with respect to Sodium
Taco Bell-Sodium is infeasible.
Solving Taco Bell with respect to Calories From Fat
Taco John's

Solving Taco John's with respect to Sugars
Taco John's-Sugars is infeasible.
Solving Taco John's with respect to Sodium
Taco John's-Sodium is infeasible.
Solving Taco John's wit

,Sugars,Sodium,Calories From Fat
Arby's,-,-,-
Baskin-Robbins,-,-,-
Blimpie,-,3082.508451,354.837983
Bojangles,-,-,715.559409
Boston Market,-,-,-
Buffalo Wild Wings,-,-,-
Burger King,-,-,-
Carl's Jr,-,-,-
Chipotle,-,-,354.95122
Culvers,-,-,652.871195


,Restaurant,Objective,Amount,Food,Sodium,Sugars,Calories From Fat,Calories,Protein,Total Carbohydrates,Dietary Fiber,Saturated Fat
0,Blimpie,Sodium,1.335890,Bagel,700mg,12g,10,290,11g,58g,3g,0.0g
1,Blimpie,Sodium,1.664110,"Bluffin, Plain",240mg,2g,10,130,5g,25g,2g,0.0g
2,Blimpie,Sodium,3.000000,Red Wine Vinegar,0mg,0g,0,5,0g,1g,0g,0.0g
3,Blimpie,Sodium,3.000000,Garden,15mg,3g,5,30,2g,6g,3g,0.0g
4,Blimpie,Sodium,0.178669,"Popcorn, large",5mg,1g,570,1180,20g,132g,23g,11.0g
...,...,...,...,...,...,...,...,...,...,...,...,...
179,McDonald's,Calories From Fat,3.000000,McDonald's Large Iced Tea,20mg,0g,0,5,1g,0g,0g,0.0g
180,McDonald's,Calories From Fat,2.816519,McDonald's Fruit & Maple Oatmeal w/o Brown Sugar,115mg,18g,40,260,6g,49g,5g,1.5g
181,McDonald's,Calories From Fat,0.523010,McDonald's Premium Southwest Salad w/ Grilled ...,1070mg,9g,100,350,37g,27g,6g,4.5g
182,McDonald's,Calories From Fat,2.118498,McDonald's Medium French Fries,230mg,0g,140,340,4g,44g,4g,2.0g


<IPython.core.display.Javascript object>

### Status
### Finished
* Overage / shortage variable convention shift to allow mixed objective relaxations.
* Comma -> Period sub
* Check whether data problems are consistent by restaurant or whatever & fix! (Ex. Godfather's Pizza)
* Nutritional Facts is absolutely referenced. This will probably need to be addressed with VBA
* Focusing on separate objectives, use continuous variables (updated to default in create_model())
* Implement more verbosity flags
* Take stock of constraints applied to models to include in paper
* Add results files to Teams drive
* Pull in new FFN restaurants
* Check whether Dr. Talcott replied to Dr. Butenko about using (Trans + Saturated Fat) * Multiplier instead of the direct Calories From Fat provided - leave it as is
* Consider making main.py dynamically fetch the list of restaurants. Would require tracking specials separately, but would allow the script to be run blankly, without people needing to know how... Done!
* Add Vitamin C to main.py downloaded quantity, even though it won't affect the model

### In Progress
* Contact FFN about dataset problems
* Work with Mykyta to host the notebook online / write readme to allow others to run
* Why no Panera?